## Requests와 BeautifulSoup 적용하기

01. Requests 라이브러리란?
특정 웹사이트에 HTTP 요청을 보내 HTML 문서를 받아올 수 있는 라이브러리이다. 근데 정확히 말하면 얘가 가져오는 HTML 문서는 문서가 아닌 그냥 단순한 String이고, BeautifulSoup에 의해 살아있는 HTML 문서로 바뀌게 된다.

02. response = requests.get(url, headers=headers) 의미?
url 주소로 GET 요청(requests.get(url))을 보냈고, 서버에서는 그 요청을 받아서 요청자인 나에게 응답을 한 것. 쉽게 말해 requests.get(url)을 통해 보낸 요청을 response라는 변수에 저장한 것이다. 이후 response.status_code를 통해 정상적인 응답을 하였는지, 아니면 비정상적인 응답을 하였는지 나에게 알려준다.

(cf) print(response.status_code) => '200' (정상응답)

03. BeautifulSoup를 활용한 Parsing
soup = BeautifulSoup(파싱할 텍스트, 'html.parsing')
soup의 자료형 : bs4.BeautifulSoup

04. bs4.BeaufitulSoup에서 엘리먼트 찾기

* 페이지의 구성에 대하여 잘 아는 경우*
 1) '.' 이용 - soup 객체를 변수처럼 이용하여 찾기
 e.g) soup.title   =>  <title>네이버 만화 &gt; 요일별  웹툰 &gt; 전체웹툰</title>
 e.g) soup.a => 첫 번째로 a태그를 가지는 element 반환
 2) 각 태그는 내부의 정보들을 key-value쌍의 딕셔너리로 관리함.
 e.g) soup.a.attrs를 출력하면 =>  {'href': '#menu' ..... } 딕셔너리의 형태가 출력
     => soup.a['href'] => '#menu'태그를 찾을 수 있음
 
* 페이지의 구성을 모르는 경우 *
    => find_all(여러개), find(1개), select, select_one 사용
    자료형 : bs4.element.ResultSet

## (예제 1) Append list를 활용한 엘리먼트 추출

In [2]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [3]:
# user agent 지정
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}

# dataframe 만들기 위한 list 생성 메서드
player_list = []

def append_list(i) :
    for info in player_info :
        score = info.find_all('td')
        number= score[0].text
        name = score[3].text
        position = score[4].text
        age = score[5].text
        nation = score[6].img['title']
        team = score[7].a['title']
        value = score[8].text.strip()
        
        player_list.append([number, name, position, age, nation, team, value])

In [6]:
# page1-4까지 엘리먼트 추출

for i in range(1,5) :
    url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={i}'
    response = requests.get(url, headers=headers)
    
    ## 파싱 및 odd, even 엘리먼트 추출
    soup = BeautifulSoup(response.text, 'html.parser')
    player_info = soup.find_all('tr', class_=['odd', 'even'])
    
    append_list(i)
    time.sleep(1)

In [7]:
df = pd.DataFrame(player_list, columns = ['number', 'name', 'position', 'age', 'nation', 'team', 'value'])

In [9]:
df

,number,name,position,age,nation,team,value
0,1,Kylian Mbappé,Centre-Forward,23,France,Paris Saint-Germain,€160.00m
1,2,Erling Haaland,Centre-Forward,21,Norway,Borussia Dortmund,€150.00m
2,3,Vinicius Junior,Left Winger,21,Brazil,Real Madrid,€100.00m
3,4,Mohamed Salah,Right Winger,29,Egypt,Liverpool FC,€100.00m
4,5,Harry Kane,Centre-Forward,28,England,Tottenham Hotspur,€100.00m
...,...,...,...,...,...,...,...
95,96,Theo Hernández,Left-Back,24,France,AC Milan,€48.00m
96,97,Luis Díaz,Left Winger,25,Colombia,Liverpool FC,€45.00m
97,98,Raphinha,Right Winger,25,Brazil,Leeds United,€45.00m
98,99,Ferran Torres,Left Winger,22,Spain,FC Barcelona,€45.00m


In [10]:
df.to_csv('market100.csv', index=False)
print('<< 파일 저장 완료 >>')

<< 파일 저장 완료 >>
